In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
6.55238884100001
GPU (s):
0.08951229400000216
GPU speedup over CPU: 73x


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist
import pandas as pd

## Functional API for DoubleDigit Classification (in-depth)

In [ ]:
# # HYPERPARAMETERS
# BATCH_SIZE = 64
# WEIGHT_DECAY = 0.001
# LEARNING_RATE = 0.001

# train_df = pd.read_csv("train.csv")
# test_df = pd.read_csv("test.csv")
# train_images = os.getcwd() + "/train_images/" + train_df.iloc[:, 0].values
# test_images = os.getcwd() + "/test_images/" + test_df.iloc[:, 0].values

# train_labels = train_df.iloc[:, 1:].values
# test_labels = test_df.iloc[:, 1:].values

# def read_image(image_path, label):
#     image = tf.io.read_file(image_path)
#     image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

#     # In older versions you need to set shape in order to avoid error
#     # on newer (2.3.0+) the following 3 lines can safely be removed
#     image.set_shape((64, 64, 1))
#     label[0].set_shape([])
#     label[1].set_shape([])

#     labels = {"first_num": label[0], "second_num": label[1]}
#     return image, labels


# AUTOTUNE = tf.data.experimental.AUTOTUNE
# train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
# train_dataset = (
#     train_dataset.shuffle(buffer_size=len(train_labels))
#     .map(read_image)
#     .batch(batch_size=BATCH_SIZE)
#     .prefetch(buffer_size=AUTOTUNE)
# )

# test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
# test_dataset = (
#     test_dataset.map(read_image)
#     .batch(batch_size=BATCH_SIZE)
#     .prefetch(buffer_size=AUTOTUNE)
# )

In [ ]:
# inputs = keras.Input(shape=(64, 64, 1))
# x = layers.Conv2D(
#     filters=32,
#     kernel_size=3,
#     padding="same",
#     kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
# )(inputs)
# x = layers.BatchNormalization()(x)
# x = keras.activations.relu(x)

# x = layers.Conv2D(64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY),)(x)
# x = layers.BatchNormalization()(x)
# x = keras.activations.relu(x)
# x = layers.MaxPooling2D()(x)

# x = layers.Conv2D(
#     64, 3, activation="relu", kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
# )(x)

# x = layers.Conv2D(128, 3, activation="relu")(x)
# x = layers.MaxPooling2D()(x)

# x = layers.Flatten()(x)
# x = layers.Dense(128, activation="relu")(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(64, activation="relu")(x)

# Because of 2 outputs Functional API makes it very easy to implement

# output1 = layers.Dense(10, activation="softmax", name="first_num")(x)
# output2 = layers.Dense(10, activation="softmax", name="second_num")(x)

# model = keras.Model(inputs=inputs, outputs=[output1, output2])

# model.compile(
#     optimizer=keras.optimizers.Adam(LEARNING_RATE),
#     loss=keras.losses.SparseCategoricalCrossentropy(),
#     metrics=["accuracy"],
# )

# model.fit(train_dataset, epochs=5, verbose=2)
# model.evaluate(test_dataset, verbose=2)

## Model Subclassing with Keras

In [ ]:
(x_train, y_train), (x_test,y_test) = mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


For each layer,  CNN -> BatchNorm -> ReLU (common structure)


---


x10 (a lot of code to write!)

So we create a subclass to directly add a layer without having to type so much


## CNN Block SubClass

In [ ]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        print(x.shape)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x


model = keras.Sequential(
    [CNNBlock(32),
     CNNBlock(64),
     CNNBlock(128),
     layers.Flatten(),
     layers.Dense(10),
    ]
)


#print(model.summary())
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=5, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/5
(None, 28, 28, 32)
(None, 28, 28, 64)
(None, 28, 28, 128)
(None, 28, 28, 32)
(None, 28, 28, 64)
(None, 28, 28, 128)
(None, 28, 28, 32)
(None, 28, 28, 64)
(None, 28, 28, 128)
938/938 - 22s - loss: 0.5381 - accuracy: 0.9479 - 22s/epoch - 23ms/step
Epoch 2/5
938/938 - 21s - loss: 0.0833 - accuracy: 0.9831 - 21s/epoch - 22ms/step
Epoch 3/5
938/938 - 21s - loss: 0.0341 - accuracy: 0.9897 - 21s/epoch - 23ms/step
Epoch 4/5
938/938 - 21s - loss: 0.0267 - accuracy: 0.9915 - 21s/epoch - 22ms/step
Epoch 5/5
938/938 - 21s - loss: 0.0266 - accuracy: 0.9914 - 21s/epoch - 23ms/step
(None, 28, 28, 32)
(None, 28, 28, 64)
(None, 28, 28, 128)
157/157 - 1s - loss: 0.0539 - accuracy: 0.9849 - 1s/epoch - 9ms/step


[0.05394566059112549, 0.9848999977111816]

ResBlock SubClass

In [ ]:
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training,)
        x = self.pooling(x)
        return x
